In [ ]:
# Screen files in folder (expects patient subfolders)

import cordialrt.screen_files.dicom_files_dataframe 

df, errors = cordialrt.screen_files.dicom_files_dataframe.open_dicom_files(parent_folder = 'D:/Anon_dicom_files/Test_dicom/dicompyler_core/') 
display(df)

In [ ]:
#Explore the content of the files thorugh the patient_object

import cordialrt.screen_files.helpers

df['planned_fractions'] = df.patient_object.apply(cordialrt.screen_files.helpers.planned_fractions)
df['main_fractions'] = df.planned_fractions.apply(cordialrt.screen_files.helpers.fractions_main_plan)

display(df [['patient_id','planned_fractions','main_fractions']])

In [ ]:
# Create a new treatment collection in the databse to represent a research project
import cordialrt.database.database as rtdb

with rtdb.DatabaseCall() as db:
    test_treatment_collection_id = db.create_treatment_collection('Treatment collection test')

print(test_treatment_collection_id)

In [ ]:
# Add a treatment to the treatment collection in the database. 
# DICOM files asscociated to the plan names, chosen will be associated with the treatment.

all_errors = list()
with rtdb.DatabaseCall() as db:
    for index, row in df.iterrows():
        status, error_log = db.add_new_treatment_to_collection_from_plan_names(patient = row.patient_object, 
                                                                collection_id =  test_treatment_collection_id,
                                                                plan_names = ['B1'],
                                                                treatment_place = 'Centre 1',
                                                                main_reference_dose = 14,
                                                                boost_reference_dose = 0,
                                                                main_dose_scale_factor = 1,
                                                                boost_dose_scale_factor = 0,
                                                                )
        all_errors.append([row.patient_id, status, error_log])



In [ ]:
# Load treatments from a treatment collection by initiating the Treatment class

import cordialrt.analysis.treatments_from_collection 

treatments = cordialrt.analysis.treatments_from_collection.init_treatments_from_collection(treatment_collection_id = test_treatment_collection_id)

In [ ]:
# Extract information from the databse and the associated DICOM files, using the treatment class
t = treatments[0]

print('Patient id:', t.patient_id)
print('Patient dicompyler DicomParser object:', t.get_dose()) # Note if multiple dosefiles are associated with the treatment, these are summed.
print('Max dose in the dose grid', t.get_max_dose())

# Please see the definition of the Treatment class for more funtionality 


In [ ]:
# Add a Region Of Interest to the treatment class

t.add_new_roi('Breast')

# This will initiate a ROI class object
breast = t.roi_by_name('Breast')

# If the ROI class object has the same name as a roi in the structure file, the dvh can be calcalted using dicompyler
breast_dvh = breast.calculate_dvh_for_roi_name()

print('Breast volume', breast_dvh.volume)
print('Breast mean dose', breast_dvh.mean)


In [ ]:
# If we want to analyse multiple treatments at once, they may have different names for the same organ. 
# We can create a synonym collection and associate that with our treatment collection.

import cordialrt.database.database as rtdb

with rtdb.DatabaseCall() as db:
    test_synonym_collection_id = db.create_synonym_collection('Test synonym collection')
    db.associate_synonym_collection_with_treatment_colection(treatment_collection_id = test_treatment_collection_id, 
                                                            synonym_collection_id = test_synonym_collection_id)


In [ ]:
# We can then add synonyms to the collection
with rtdb.DatabaseCall() as db: 
    db.add_synoym_to_synonym_colection(test_synonym_collection_id, 'breast', 'ctvp_breast')
    db.add_synoym_to_synonym_colection(test_synonym_collection_id, 'mamma', 'ctvp_breast')
    db.add_synoym_to_synonym_colection(test_synonym_collection_id, 'mam', 'ctvp_breast')

In [ ]:
# We can now just use the standard name.

t.add_new_roi('ctvp_breast')

breast = t.roi_by_name('ctvp_breast')

# This function will look for all synonyms associated with the ROI name (for this particular treatment collection)
breast_dvh = breast.calculate_dvh_for_priority_synonym()

print('Breast volume', breast_dvh.volume)
print('Breast mean dose', breast_dvh.mean)

